<a href="https://colab.research.google.com/github/MohanPadala/gen-ai-project/blob/main/AnimeMovieRecommendSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain langchain-community langchain_groq chromadb gradio

In [ ]:
import pandas as pd
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_groq import ChatGroq

In [ ]:
anime = pd.read_csv("/content/anime_with_synopsis.csv")
anime.head()

In [ ]:
anime.dropna()

In [ ]:
anime['combined_df'] = anime.apply(lambda row: f"Title: {row['Name']}. Overview: {row['sypnopsis']} Genres: {row['Genres']}", axis=1)


In [ ]:
anime['combined_df'][0]

In [ ]:
anime[['combined_df']].to_csv('anime_updated.csv', index = False)

In [ ]:
pd.read_csv("/content/anime_updated.csv")

**Data** Loader and VectorStore using **Langchain**

In [ ]:
loader=CSVLoader(file_path="/content/anime_updated.csv")
data = loader.load()

text_splitter=CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
texts = text_splitter.split_documents(data)
embeddings = HuggingFaceEmbeddings()
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.3-70b-versatile",
    temperature=0
)






docssearch = Chroma.from_documents(texts, embeddings)

**Querying Vector DB Store for movie recommendation**

In [ ]:
query = "Iam looking for an animated sci-fi movie. What could you suggest movie"
docs=docssearch.similarity_search(query,k = 1)

In [ ]:
docs

**Using QA Retrieval for movie recommendation**

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=docssearch.as_retriever(),
    return_source_documents=True
)


In [ ]:
query = "Can you recommend a romance anime with a unique storyline?"
result=qa({"query":query})

In [ ]:
result['result']

In [ ]:
result['source_documents'][0]

**Prompt Engineering**

In [ ]:
from langchain.prompts import PromptTemplate

template = """You are a movie recommender system that help users to find anime that match their preferences.
Use the following pieces of context to answer the question at the end. For each question, suggest three anime, with a short description of the plot and the reason why the user migth like it.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Your response:"""

PROMPT = PromptTemplate(
    template=template, input_variables=["context","question"]
)
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=docssearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs)

query = "Give me a list of comedy animes that are lighthearted and fun."
result=qa({"query":query})
print(result['result'])


**Second Template**

In [ ]:
from langchain.prompts import PromptTemplate

template_prefix = """You are a movie recommender system that helps users find anime that match their preferences. Use the following pieces of context to answer the question at the end. For each question, take into account the context and the personal information provided by the user. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
"""

user_info = """This is what we know about the user, and you can use this information to better tune your research: Age: {age} Gender: {gender}
"""

template_suffix = """Question: {question}
Your response:"""

# Format user_info
user_info_filled = user_info.format(age=18, gender="female")

# Combine all parts
COMBINED_PROMPT = template_prefix + '\n' + user_info_filled + '\n' + template_suffix

print(COMBINED_PROMPT)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
import gradio as gr

# Define prompt
template_prefix = """You are a movie recommender system that helps users find anime that match their preferences. Use the following pieces of context to answer the question at the end. For each question, take into account the context and the personal information provided by the user. If you don't know the answer, just say that you don't know, don't try to make up an answer.
Context: {context}
"""

user_info = """This is what we know about the user, and you can use this information to better tune your research: Age: {age} Gender: {gender}
"""

template_suffix = """Question: {question}
Your response:"""

user_info_filled = user_info.format(age=18, gender="female")
COMBINED_PROMPT = template_prefix + '\n' + user_info_filled + '\n' + template_suffix

PROMPT = PromptTemplate(
    template=COMBINED_PROMPT, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(
    llm,
    chain_type="stuff",
    retriever=docssearch.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

def recommend_anime(query):
    result = qa({"query": query})
    return result['result']

# Gradio interface
iface = gr.Interface(
    fn=recommend_anime,
    inputs=gr.Textbox(label="Enter your anime preference"),
    outputs=gr.Textbox(label="Recommended Anime"),
    title="Anime Movie Recommender",
    theme="Respair/Shiki@1.2.1"
)

if __name__ == "__main__":
    iface.launch()
